# Library Demo

## Descriptions of IPI Functions

This notebook aims to show how to use and what functions we have created to aid with analysis in the IPI dataset.

Future users of the IPI dataset should benefit from these descriptions.

## Notebook Contents:
### - Program Structure
### - Loading Data Functions
### - Supporting and Processing Functions


## Program Structure

Our files have been organized the following layout:

**Jupyter Notebooks**
 - Library Demo (this file)
 - Final Predictions (our modelling process)
 - Plot Demo (visual representations of data)
 
**Support Functions**
 - Python library file: ___init__.py_
 - Data loading code: _load_data.py_
 - Graphing code: _plotting_funcs.py_
 - Additional Funcs: _supporting_funcs.py_
 
 
 
**Data Files**
 - Original Excel file: _Idaho_Municipal_Database_03052019.xlsx_
 - GPS Coordinates by City/ZIP: _gps_data.csv_
 - Employment Data File: _emp_data.csv_
 - Fully Compiled File: _ipi_final.csv_ << This can be remade by calling load.all_data()
 - IPI Column Descriptions: _col_only.csv_ full length descriptions
 - CPI Inflation Adjustment Data:  _bls_cpi_stats.xlsx_
 - Top 59 cities with the best data: _best_cities.csv_
 
**Data Folders**
 - GPS/  contains files needed join gps/zip/citynames
 - employment/ contains files with employment data from BLS
 
 
 
 

## Import Libraries

In [1]:
## Supporting Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Created IPI Libraries
import support.load_data as load
import support.supporting_funcs as funcs
import support.plotting_funcs as plotipi

# Library Descriptions

## Getting Data

### ALL Data
This is the most complete way to get data we used in this project

- this data is adjusted for inflation - nominal dollars to Oct 2019 dollars using cpi
- it includes GPS, employment, and IPI data
- it uses the short names
- an option to include normalized data

Running it we see output descriptions:


In [2]:
ipi_data = load.all_data(out=True,norm=True)
# note: setting out=True will write this data to ipi_final.csv


#ipi_data = pd.read_csv("ipi_final.csv") -use this once the ipi_final.csv is created

Loading IPI data
Adjusting for Inflation
Getting GPS
Getting Employees
Merge Everything
Normalize Columns
Categorize City Size
writing file to ipi_final.csv


As you can see below this includes all cities and all years

In [3]:
ipi_data.head()

,Name,County,Year4,YearofData,SurveyYr,ID,IDChanged,Type_Code,County,FIPS_Code_State,...,Counties - Burglary- Total_100k,Counties - Manslaughter- Total_100k,Counties - Murder- Total_100k,Counties - Rape-Total_100k,Counties - Robbery- Total_100k,N of Registered Organizations_100k,N of Orgs Filing Form 990_100k,Total_Expenditure_100k,Total_Revenue_100k,size
0,BOISE CITY,Ada,1996,96.0,96.0,132001001.0,0.0,2.0,Ada,16.0,...,1723.223580,5.237762,4.583041,51.722896,45.175694,454.375822,145.347886,124946.494324,130437.172830,urban
1,KUNA CITY,Ada,1996,96.0,96.0,132001003.0,0.0,2.0,Ada,16.0,...,93499.111901,284.191829,248.667851,2806.394316,2451.154529,24653.641208,7886.323268,54420.553644,53837.892470,non-urban
2,MERIDIAN CITY,Ada,1996,95.0,96.0,132001004.0,0.0,2.0,Ada,16.0,...,12759.974790,38.784118,33.936103,382.993164,334.513017,3364.522228,1076.259272,52528.739504,59263.804954,non-urban
3,BOISE CITY,Ada,1997,97.0,97.0,132001001.0,0.0,2.0,Ada,16.0,...,1525.498078,0.654720,9.166083,70.709782,48.449295,485.147672,156.478129,149601.991377,138227.629401,urban
4,EAGLE CITY,Ada,1997,97.0,97.0,132001801.0,0.0,2.0,Ada,16.0,...,35426.486240,15.204501,212.863007,1642.086057,1125.133039,11266.534894,3633.875627,27280.016597,29132.814865,non-urban


### Abbreviated Data
Same as all except for:
- it only includes 4 years and 59 cities
- it only includes the most important columns that we have determined

In [4]:
abb_data = load.ipi_abb()
abb_data.head()

note this requires ipi_final.csv to be created (call all_data(out=True)


,Name,County,Year4,YearofData,SurveyYr,ID,IDChanged,Type_Code,County.1,FIPS_Code_State,...,Counties - Manslaughter- Total_100k,Counties - Murder- Total_100k,Counties - Rape-Total_100k,Counties - Robbery- Total_100k,N of Registered Organizations_100k,N of Orgs Filing Form 990_100k,Total_Expenditure_100k,Total_Revenue_100k,size,DataCount
0,BOISE CITY,Ada,1997,97.0,97.0,132001001.0,0.0,2.0,Ada,16.0,...,0.654720,9.166083,70.709782,48.449295,485.147672,156.478129,149601.991377,138227.629401,urban,4
1,BOISE CITY,Ada,2002,NaN,2.0,132001001.0,0.0,2.0,Ada,16.0,...,0.538251,3.229505,87.196628,60.284089,509.185250,173.316755,153065.175562,135228.938273,urban,4
2,BOISE CITY,Ada,2007,NaN,7.0,132001001.0,NaN,2.0,Ada,16.0,...,0.503428,9.061710,103.202811,46.818836,611.162013,237.618180,128140.636956,152203.168916,urban,4
3,BOISE CITY,Ada,2012,NaN,12.0,132001001.0,NaN,2.0,Ada,16.0,...,0.000000,0.972427,59.318037,39.383287,645.691420,255.748258,147318.465719,148231.314079,urban,4
4,GARDEN CITY,Ada,1997,NaN,97.0,132001002.0,0.0,2.0,Ada,16.0,...,11.475786,160.661005,1239.384898,849.208171,8503.557494,2742.712876,90290.256736,78293.263177,non-urban,4


### Column Descriptions
the columns in this data set were large so we shortened them. We kept the descriptions in the col_only.csv
Here is an example of looking at the description of any columns with 'assault' in the name. It also returns the short names that can be used in the file.

In [5]:
ipi_cols = load.cols()
ipi_cols.tail()

,Index,LongName,ShortName
613,613,"N of Orgs Filing Form 990: Note: ""Includes org...",N of Orgs Filing Form 990
614,614,Business Establishments: Source: Census Bureau...,Business Establishments
615,615,Population Density // RAND State Statistics =>...,Population Density
616,616,"Personal income per capita // Unit: dollars, c...",Personal income per capita
617,617,Population2: Current Population Estimates,Population2: Current Population Estimates


### Additional Loading Functions
Many of these are sub functions to the all and abb ones. Here are some brief descriptions:

- load.empl() : compile employee data only 
- load.emp() : load already compiled employee data
- load.gps() : load gps data

# Processing Functions

besides the searching functions, the additional functions are automatically called by the load.all_data() call.

The examples/description serve to explain why we use them.

### Search All
This function searches the large dataframe (with all data) so we can select columns. It also creates a list of the found columns for future pandas use.

TIP: use regular expressions & capitals dont matter

In [6]:
funcs.search_all(ipi_data,'total.*revenue')

Total_Revenue
Total_IG_Revenue
Total_Fed_IG_Revenue
Total_State_IG_Revenue
Total_Utility_Revenue
Total_IG_Revenue_PerRev
Total_Fed_IG_Revenue_PerRev
Total_State_IG_Revenue_PerRev
Total_Utility_Revenue_PerRev
Total_Revenue_100k


array(['Total_Revenue', 'Total_IG_Revenue', 'Total_Fed_IG_Revenue',
       'Total_State_IG_Revenue', 'Total_Utility_Revenue',
       'Total_IG_Revenue_PerRev', 'Total_Fed_IG_Revenue_PerRev',
       'Total_State_IG_Revenue_PerRev', 'Total_Utility_Revenue_PerRev',
       'Total_Revenue_100k'], dtype=object)

## Search Column Detail

search_column()
- as seeen below searches for column descriptions from the short names (we have shortened them for ease of use)


In [7]:
funcs.search_column('assault',ipi_cols,disp=True) # first argument is a regex search function

Assault, Total - Municipalities // Source: RAND => Crimes, Arrests, & Clearances (Description: "This database reports the number of offenses (murder and nonnegligent manslaughter, forcible rape, robbery, aggravated assault, burglary, larceny-theft, motor vehicle theft, and arson) and auxiliary offense data (e.g., burglary: forcible entry, unlawful entry, and attempted entry) for each agency reporting data to the Uniform Crime Reporting Program (UCR) in U.S. States. See crime definitions for crime descriptions. 
The database also reports the number of reported offenses cleared by arrest or exceptional means and the number of clearances which involved only juveniles (under 18 years of age). The category "total offenses cleared by an arrest" do not contain data for the years 1960-1963."; Originating source: Federal Bureau of Investigation; Unit: municipalities) => Area: Cities in Idaho => Offense: Assault total, Burglary total, Manslaughter, Murder, Rape total, Robbery total, Total all th

array(['Assault- Total - Municipalities', 'Counties - Assault- Total'],
      dtype=object)

# Normalization

There are 3 main data types that could potentially benefit from some adjustment/normalization:
This funcs.normalize() function creates new variables to work with.


- Expenditure: we have various expenditure categories [what the city spent] (as well as total expenditure)
    The normalization will divide each expenditure category by the total expenditure and multiply by 100
        This gives us the percent of each expenditure category out of the whole

    _Columns with this norm have **PerExp** at the end of the name_
        
        
- Revenue: we have various revenue categories [money the city made] (as well as total revenue)
    The normalization will divide each revenue category by the total revenue and multiply by 100
        This gives us the percent of each revenue category out of the whole
        
    _Columns with this norm have **PerRev** at the end of the name_
    

- Crime/#of city employees: this data is measured in counts.
        We thought it would be useful to have the rates per 100,000 people 
        (which is how many organizations report crime)
    This normalization divides each crime/#employee data by the population _at that time_ and multiplies by 100,000
        
    _Columns with this norm have **Per100k** at the end of the name_

### Norm Example
Below is an example of each of the three columns
- (total current hospital expenditure / total expenditure)*100
- (total water utility revenue / total revenue)*100
- (city assault counts / population)*100,000

In [8]:
cols = funcs.search_all(ipi_data,'name|assault.*munic.*100k|water.*utility.*perrev|total_hospital.*current.*perexp')

Name
Total_Hospital_Current_Exp_PerExp
Water_Utility_Revenue_PerRev
Assault- Total - Municipalities_100k


In [9]:
ipi_data[cols].head()

,Name,Total_Hospital_Current_Exp_PerExp,Water_Utility_Revenue_PerRev,Assault- Total - Municipalities_100k
0,BOISE CITY,0.0,0.000000,1880.356430
1,KUNA CITY,0.0,27.164502,NaN
2,MERIDIAN CITY,0.0,17.187710,1279.875891
3,BOISE CITY,0.0,0.000000,1693.106451
4,EAGLE CITY,0.0,3.430962,NaN


## Categorize Size

As another useful metric, we categorized the city size based on census rules:
- urban: >50,000 people
- rural: < 2,500 people
- non-urban: < 50,000 people AND > 2,500 people


This funcs.categorize_size() function creates a 'size' column in the data.

In [10]:
ipi_data[['Name','size']].head()

,Name,size
0,BOISE CITY,urban
1,KUNA CITY,non-urban
2,MERIDIAN CITY,non-urban
3,BOISE CITY,urban
4,EAGLE CITY,non-urban


## Adjust For Inflation

The money portions money of data we were given was for the year recorded, so we felt it was necessary to adjust the dollar amounts to match current-day amounts. After this function runs, all money columns are adjusted to match October-2019 amounts.

In [11]:
orig_data =  pd.read_excel('Idaho_Municipal_Database_03052019.xlsx', header=1)


In [12]:
print("Un-adjusted")
print(orig_data[['Name','Year4','Total_Expenditure']].head())

print("\n\n After Adjustment\n")
boise = ipi_data.loc[ipi_data['Name'] == "BOISE CITY"].sort_values('Year4')
print(boise[['Name','Year4','Total_Expenditure']].head())

Un-adjusted
         Name  Year4  Total_Expenditure
0  BOISE CITY   1996           116352.0
1  BOISE CITY   1997           142508.0
2  BOISE CITY   1998           147094.0
3  BOISE CITY   1999           148123.0
4  BOISE CITY   2000           157609.0


 After Adjustment

          Name  Year4  Total_Expenditure
0   BOISE CITY   1996      190839.527036
3   BOISE CITY   1997      228497.593570
8   BOISE CITY   1998      232233.451067
12  BOISE CITY   1999      228804.691224
16  BOISE CITY   2000      235540.335157
